In [1]:
import os
os.sys.path.append("retinaface")

In [2]:
from detector import Detector
from openvino.inference_engine import IECore
import cv2
import numpy as np

In [3]:
model_path = "/media/prateek/prateek_space/intel_model_dump/public/retinaface-resnet50/openvino_model/R50-0000.xml"
face_thresh = .5

In [4]:
channel = "/home/prateek/prateek_space/dataset/veraval_production/videos/face_veraval/202003211445.mp4"
cap  = cv2.VideoCapture(channel)

ie = IECore()
detector = Detector(ie,model_path, face_thresh, "MULTI:CPU,GPU")

In [5]:
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
while True:
    ret,frame = cap.read()
    if not ret:
        break
    detection, detect_masks = detector.detect(frame)
    for i, (score, xmin, ymin, xmax, ymax) in enumerate(zip(*detection.face_detection)):
        xmin = int(max(0, xmin))
        ymin = int(max(0, ymin))
        xmax = int(min(frame.shape[1], xmax))
        ymax = int(min(frame.shape[0], ymax))
        color = (255, 0, 0)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(frame, str(round(score * 100, 1)) + ' %', (xmin, ymin - 7),
                     cv2.FONT_HERSHEY_COMPLEX, 0.6, color, 1)
        for j in range(5):
            x = detection.landmarks_regression[0][i, j].astype(np.int)
            y = detection.landmarks_regression[1][i, j].astype(np.int)
            cv2.circle(frame, (x, y), 2, (0, 255, 255), 2)

    cv2.putText(frame, 'summary: {:.1f} FPS'.format(
        1.0 / detector.infer_time), (5, 15), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 200))
    cv2.imshow("preview",frame)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break


cv2.destroyAllWindows()
cap.release()